In [24]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [25]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(128, (3, 3), input_shape=(100,100,3)))#Filtre sayısı,Filtre penceresi(kernel) boyutu,Giriş görüntüsü değerleri
model.add(Activation('relu')) #Aktivasyon fonksiyonu
model.add(MaxPooling2D(pool_size=(2, 2))) #Konvolüsyon katmanı için giriş boyutunu azaltmaktır.Ezberlemenin önüne geçmek için

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten()) #Bu sinir ağındaki verileri tek boyutlu diziye çeviriyor.
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5)) # Ağın ezber yapmasının önüne geçmesi için bu katman kullanılır.
model.add(Dense(1,activation="sigmoid"))

model.compile(loss='binary_crossentropy',optimizer='rmsprop', metrics=['accuracy'])#Hata değeri,ağın güncellemesi,model değerlendşrme

In [26]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 98, 98, 128)       3584      
_________________________________________________________________
activation_23 (Activation)   (None, 98, 98, 128)       0         
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 49, 49, 128)       0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 47, 47, 32)        36896     
_________________________________________________________________
activation_24 (Activation)   (None, 47, 47, 32)        0         
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 21, 21, 64)       

In [27]:
batch_size = 16
train_datagen = ImageDataGenerator(
        rescale=1./255,#Görüntüyü yeniden ölçeklemek için
        shear_range=0.2,#Kaydırma için kullanılır
        zoom_range=0.2,#Yakınlaştırma ve uzaklaştırma için
        horizontal_flip=True)#Görüntüyü çevirmek için
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        'data/train',  
        target_size=(100, 100),  #Görüntülerin boyutu
        batch_size=batch_size,#Veri yığınları boyutu
        class_mode='binary') 
validation_generator = test_datagen.flow_from_directory(
        'data/validation',
        target_size=(100, 100),
        batch_size=batch_size,
        class_mode='binary')

Found 2002 images belonging to 2 classes.
Found 433 images belonging to 2 classes.


In [28]:
print(validation_generator.class_indices)

{'CloseFace': 0, 'OpenFace': 1}


In [29]:
model.fit_generator(
        train_generator,
        steps_per_epoch=2002 // batch_size, #Toplu yineleme sayısı eğitim bitmeden 
        epochs=50,#Model eğitilirken modelden veilerin kaç kez geçiş yapacağı
        validation_data=validation_generator,
        validation_steps=433 // batch_size)

Epoch 1/50
125/125 [==============================] - 85s 680ms/step - loss: 0.7049 - accuracy: 0.5373 - val_loss: 0.7016 - val_accuracy: 0.6713
Epoch 2/50
125/125 [==============================] - 89s 714ms/step - loss: 0.6822 - accuracy: 0.5866 - val_loss: 0.6188 - val_accuracy: 0.6715
Epoch 3/50
125/125 [==============================] - 89s 712ms/step - loss: 0.6275 - accuracy: 0.6506 - val_loss: 0.4488 - val_accuracy: 0.7818
Epoch 4/50
125/125 [==============================] - 91s 730ms/step - loss: 0.5531 - accuracy: 0.7412 - val_loss: 0.3639 - val_accuracy: 0.8465
Epoch 5/50
125/125 [==============================] - 90s 718ms/step - loss: 0.4673 - accuracy: 0.7885 - val_loss: 0.2023 - val_accuracy: 0.8729
Epoch 6/50
125/125 [==============================] - 91s 732ms/step - loss: 0.4037 - accuracy: 0.8248 - val_loss: 0.4616 - val_accuracy: 0.8897
Epoch 7/50
125/125 [==============================] - 90s 717ms/step - loss: 0.3238 - accuracy: 0.8570 - val_loss: 0.1553 - val_ac

In [30]:
model.save_weights('model_generator_weight.h5')  
model.save('model_generator.h5')